## **Elaborazione di Immagini Mediche**
### Contest 2021/22 - Segmentazione ghiandola prostatica in immagini MRI

### Script di Testing

Rigazio Sofia, Roccaro Lucia, Romano Anastasio, Ruzzante Elena

* Installazione delle librerie necessarie

In [ ]:
# Before running the script, reset the runtime to factory reset (Runtime -> Factory Reset Runtime)
# and then change runtime type to GPU (Runtime -> Change runtime type)

!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install segmentation_models==1.0.1
!pip install h5py==2.10.0 
!pip install plotly==5.3.1



*   Collegamento a Google Drive e import delle librerie necessarie



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import random
import numpy as np
import plotly.express as px

from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage.io import imread, imshow, imsave
from skimage.transform import resize
from skimage.segmentation import mark_boundaries

from keras.models import load_model

# aggiunti
from tifffile import imsave as imsave_tiff
from scipy import ndimage
from skimage import morphology

* Caricamento dataset in formato .rar da Google Drive a Colab

In [ ]:
!pip install unrar
!unrar x "drive/MyDrive/Colab Notebooks/EIM/Contest 2021-22/DATASET_stu.rar" "drive/MyDrive/Colab Notebooks/EIM/Contest 2021-22/"

*   Preparazione delle cartelle

In [ ]:
current_dir = 'drive/MyDrive/Colab Notebooks/EIM/Contest 2021-22/'
dataset_name = 'DATASET_stu'

# Path
TRAIN_IMG_path = os.path.join(current_dir,dataset_name,'train','images')
TRAIN_MASK_path = os.path.join(current_dir,dataset_name,'train','manual')
TRAIN_AUTO_path = os.path.join(current_dir,dataset_name,'train','automatic')

VAL_IMG_path = os.path.join(current_dir,dataset_name,'val','images')
VAL_MASK_path = os.path.join(current_dir,dataset_name,'val','manual')
VAL_AUTO_path = os.path.join(current_dir,dataset_name,'val','automatic')

TEST_IMG_path = os.path.join(current_dir,dataset_name,'test','images')
TEST_AUTO_path = os.path.join(current_dir,dataset_name,'test','automatic')

# Creazione delle cartelle che conterranno i risultati della segmentazione automatica
try:
  os.mkdir(TRAIN_AUTO_path)
  os.mkdir(VAL_AUTO_path)
  os.mkdir(TEST_AUTO_path)
except:
  pass

* Estrazione delle liste di volumi

In [ ]:
# Training set
train_images = os.listdir(TRAIN_IMG_path)
n_images_train = len(train_images)

# Validation set
val_images = os.listdir(VAL_IMG_path)
n_images_val = len(val_images)

# Test set
test_images = os.listdir(TEST_IMG_path)
n_images_test = len(test_images)

* Lettura di un'immagine e della rispettiva maschera e settaggio dei parametri

In [ ]:
# Lettura di un generico volume
chosen_image_name = train_images[0] # prendiamo la prima immagine
original_volume = imread(os.path.join(TRAIN_IMG_path, chosen_image_name))

# definiamo quindi i parametri
n_slices = original_volume.shape[0]
IMG_WIDTH = original_volume.shape[2] # la larghezza sono le colonne
IMG_HEIGHT = original_volume.shape[1] # l'altezza sono le righe
NUM_CLASSES = 2 # vogliamo segmentare due classi: prostata e background
IMG_CHANNELS = 3 # Le immagini fornite sono grayscale, per poter utilizzare i pesi preallenati di ImageNet forzeremo le immagini a RGB

* Definizione di una funzione che effettua il preprocessing

In [ ]:
def preprocess(image):

  # RIMOZIONE DEL BORDO NERO (SE PRESENTE)
  # rimuoviamo le colonne completamente nere
  idx = np.argwhere(np.all(image == 0, axis=0))
  image = np.delete(image, idx, axis=1)
  # rimuoviamo le righe completamente nere
  idx = np.argwhere(np.all(image == 0, axis=1))
  image = np.delete(image, idx, axis=0)
  # riportiamo alla dimensione originale (solo se ne abbiamo modificato le dimensioni)
  if image.shape != (IMG_HEIGHT,IMG_WIDTH):
    image = resize(image, (IMG_HEIGHT,IMG_WIDTH), preserve_range=True).astype(np.uint8)

  # filtro gaussiano
  def kernel_gauss(size, sigma):
    v = np.linspace(-(size-1)/2,(size-1)/2,size)
    x, y = np.meshgrid(v,v)
    h = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    h = h/h.sum()
    return h

  image = ndimage.correlate(image,kernel_gauss(3,3))

  return image

* Definizione di una funzione che effettua il post-processing

In [ ]:
def postprocess(image,mask):

  # REINSERIMENTO DEL BORDO NERO NELLA MASCHERA (SE PRESENTE NELL'IMMAGINE ORIGINALE)
  idx_cols = np.argwhere(np.all(image == 0, axis=0))
  idx_rows = np.argwhere(np.all(image == 0, axis=1))

  # riportiamo alla dimensione originale (solo se ne abbiamo modificato le dimensioni)
  if (len(idx_cols) != 0) or (len(idx_rows)!= 0):
    mask = resize(mask, (IMG_HEIGHT-len(idx_rows),IMG_WIDTH-len(idx_cols)), preserve_range=True).astype(np.float32)
  # inseriamo le colonne nere
  for i in idx_cols:
    mask = np.insert(mask,i,0.0,axis=1)
  # inseriamo le righe nere
  for i in idx_rows:
    mask = np.insert(mask,i,0.0,axis=0)

  # Rimozione di piccoli oggetti e piccoli buchi
  mask = morphology.remove_small_holes(mask.astype(bool))
  mask = morphology.remove_small_objects(mask)

  # torniamo al formato originale -> uint8
  mask = mask.astype(np.uint8)
  
  return mask

* Load del modello allenato

In [ ]:
net_name = 'trained_net.h5'
model_path = os.path.join(current_dir, net_name)
model = load_model(model_path)

* Creazione delle maschere automatiche

In [ ]:
def net_prediction(list_images,IMG_path,AUTO_path):
  for n, id_ in tqdm(enumerate(list_images), total=len(list_images)):
    # la variabile "n" rappresenta un contatore (0-num_immagini) mentre "id_" 
    # contiene il nome della n-esima immagine

    # Load dei dati
    image = imread(IMG_path+'/'+id_)
    
    # creazione matrice vuota che conterrà le maschere del volume da salvare
    mask_auto_volume = np.zeros((n_slices,IMG_WIDTH,IMG_HEIGHT), dtype=np.uint8) 

    for i in range(n_slices): # consideriamo una slice alla volta
      img_slice = image[i]

      # Preprocessing
      img_slice = preprocess(img_slice)

      # Forziamo l'immagine a RGB impilando 3 layer contenenti gli stessi valori
      img_slice = np.reshape(img_slice,(1,IMG_WIDTH,IMG_HEIGHT))
      img_slice = np.stack((img_slice,img_slice,img_slice), axis = 3)
      
      
      # Apply CNN for prediction
      softmax = model.predict(img_slice)
      softmax = np.squeeze(softmax)

      mask_auto = softmax[:,:,1]

      # scegliamo soglia pari a 0.5 in modo da massimizzare la distanza tra i valori attesi (0 e 1)
      soglia = 0.5
      mask_auto[mask_auto<soglia] = 0
      mask_auto[mask_auto>=soglia] = 1

      # Post-processing
      mask_auto_volume[i] = postprocess(image[i],mask_auto)

    # Save mask auto
    imsave_tiff(AUTO_path+'/'+id_, mask_auto_volume)

In [ ]:
# Training Set
print('Reading images - Training set')
net_prediction(train_images,TRAIN_IMG_path,TRAIN_AUTO_path)

# Validaion Set
print('Reading images - Validation set')
net_prediction(val_images,VAL_IMG_path,VAL_AUTO_path)

# Test Set
print('Reading images - Test set')
net_prediction(test_images,TEST_IMG_path,TEST_AUTO_path)

* Visualizzazione dei risultati

In [ ]:
random_index = random.randint(0,n_images_train-1)
image = imread(TRAIN_IMG_path+'/'+ train_images[random_index])
mask_auto = imread(TRAIN_AUTO_path+'/'+ train_images[random_index])
mask_manual= imread(TRAIN_MASK_path+ '/' + train_images[random_index])

# Rappresentazione a video del volume prostatico
fig = px.imshow(image, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

# Rappresentazione a video della corrispondente maschera binaria automatica
fig = px.imshow(mask_auto, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

# Rappresentazione a video della corrispondente maschera binaria manuale
fig = px.imshow(mask_manual, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

In [ ]:
# TEST SET
random_index = random.randint(0,n_images_test-1)
image = imread(TEST_IMG_path+'/'+ test_images[random_index])
mask_auto = imread(TEST_AUTO_path+'/'+ test_images[random_index])

# Rappresentazione a video del volume prostatico
fig = px.imshow(image, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

# Rappresentazione a video della corrispondente maschera binaria automatica
fig = px.imshow(mask_auto, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

* Creazione immagini Test Set con maschera sovrapposta 

In [ ]:
# Per una più agevole visualizzazione il codice crea automaticamente una cartella 
# ('TEST_recap') in cui sono state inserite le sovrapposizioni frame per frame 
# di maschera e immagine originale per tutte le 10 immagini del Test Set.

# creazione cartella 
TEST_recap = os.path.join(current_dir,'TEST_recap')
try:
  os.mkdir(TEST_recap)
except:
  pass

for index in range(0, n_images_test):
  image = imread(TEST_IMG_path+'/'+ test_images[index])
  mask_auto = imread(TEST_AUTO_path+'/'+ test_images[index])

  fig, axs = plt.subplots(6, 4,figsize=(25,40))
  fig.suptitle(f'Image: {test_images[index]}')

  for i in range(0, n_slices):
    row = int(i/4)
    col = int(i-4*row)

    axs[row, col].imshow(mark_boundaries(image[i,:,:],mask_auto[i,:,:],color=(1,0,0), mode='thick')), # inserisce il bordo
    axs[row, col].imshow(mask_auto[i,:,:], 'OrRd', interpolation='none', alpha=0.2) # evidenzia in rosso dove la maschera ha selezionato
    axs[row, col].set_title(f'slice {i+1}')

  # salvo l'immagine
  plt.savefig(TEST_recap+'/T_OW_'+test_images[index].split('.')[0]+'.png')

* Calcolo del coefficiente di Dice Similarity

In [ ]:
def calculate_DSC(MASK_path, AUTO_path, list_images):

  n_images = len(list_images)
  DSC_set = np.zeros(n_images, dtype = np.float64)
  
  for i in range(0,n_images):
    id_ = list_images[i]
    x = imread(MASK_path+'/'+ id_) # manual mask
    y = imread(AUTO_path+'/'+ id_) # automatic mask

    x_int_y = np.logical_and(x,y)
    
    num = 2.*x_int_y.sum()
    den = x.sum()+y.sum()

    DSC_set[i] = num/den

  return DSC_set	

In [ ]:
# Training Set
DSC_train = calculate_DSC(TRAIN_MASK_path, TRAIN_AUTO_path, train_images)
DSC_train_avg = np.mean(DSC_train)
# Validation Set
DSC_val = calculate_DSC(VAL_MASK_path, VAL_AUTO_path, val_images)
DSC_val_avg = np.mean(DSC_val)

print(f"DSC training = {DSC_train_avg}")
print(f"DSC validation = {DSC_val_avg}")

* Grafico DSC su Training e Validation Set

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.suptitle('Dice Similarity Coefficient')

# Training Set
ax1 = fig.add_subplot(1,2,1)
ax1.scatter(range(1,n_images_train+1),DSC_train)
ax1.set_ylim([0, 1])
ax1.set_title('Training Set')
ax1.set_ylabel('DSC')
ax1.set_xlabel('Numero volume')

# Validation Set
ax2 = fig.add_subplot(1,2,2)
ax2.scatter(range(1,n_images_val+1),DSC_val)
ax2.set_ylim([0, 1])
ax2.set_title('Validation Set')
ax2.set_ylabel('DSC')
ax2.set_xlabel('Numero volume')